<a href="https://colab.research.google.com/github/TrinaDutta95/Analyzing-hallucinations-in-machine-summarization/blob/main/Understanding_BART_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.2 MB/s eta 0:00:00


Loading input text

In [ ]:
file = open("e19.txt", "r")
text = file.read()
print(text)

The winners of a TV talent show have beaten the Pope to the album of the year at the Classical Brit Awards.
Only Men Aloud, winners of the BBC's Last Choir Standing, won with Band of Brothers, beating Alma Mater's Music From The Vatican, featuring the Pope.
Other winners included conductor Vasily Petrenko and soprano Angela Gheorghiu.
Soprano Kiri Te Kanawa was honoured with a lifetime achievement award at the ceremony hosted by Myleene Klass at London's Royal Albert Hall.
The New Zealand-born singer joins a list of past recipients which includes Placido Domingo and Jose Carreras.
The soundtrack of the year was awarded to the Oscar-nominated Revolutionary Road by American composer Thomas Newman,  while Thomas Ades won composer of the year for the Tempest.
Only Men Aloud's win saw them beat 14-year-old Faryl Smith, a finalist on Britain's Got Talent in 2008.
Teenager Smith also lost out in the young British classical performer category to violinist Jack Liebeck.
The critics' choice awar

Loading model

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

#initializing BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-xsum")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-xsum")

inputs = tokenizer([text], max_length=1024, truncation=True, return_tensors="pt")

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Encoder and decoder representation:

In [ ]:
the_encoder = model.get_encoder()
the_decoder = model.get_decoder()
last_linear_layer = model.lm_head

In [ ]:
input_representation = the_encoder(input_ids = inputs.input_ids,
                                   attention_mask = inputs.attention_mask)
print(input_representation)

BaseModelOutput(last_hidden_state=tensor([[[-3.5447e-03,  3.9355e-02,  2.6076e-02,  ...,  7.4530e-03,
           4.3769e-03,  3.5721e-03],
         [-8.0658e-02, -1.0385e-01,  2.4355e-03,  ..., -1.3961e-01,
           4.7018e-02, -8.3029e-02],
         [-1.8983e-02, -2.9701e-01,  2.5846e-01,  ..., -1.0105e-01,
          -1.4746e-01, -1.7797e-01],
         ...,
         [ 1.9296e-01, -6.3843e-01, -5.2633e-01,  ...,  2.9090e-02,
           2.0575e-01,  1.5995e-01],
         [-6.8581e-04,  1.0773e-02, -4.2279e-04,  ...,  1.0172e-02,
           1.0077e-02, -3.7954e-03],
         [ 8.6688e-02,  3.2625e-01,  9.9251e-02,  ..., -9.4603e-03,
          -2.1331e-02,  3.4963e-02]]], grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


In [ ]:
import torch

start_token = torch.tensor( [tokenizer.bos_token_id] ).unsqueeze(0)
mask_ids    = torch.tensor( [1] ).unsqueeze(0)

decoder_output = the_decoder(input_ids=start_token,
                             attention_mask=mask_ids,
                             encoder_hidden_states=input_representation[0],
                             encoder_attention_mask=inputs.attention_mask )
decoder_output = decoder_output.last_hidden_state

logits = last_linear_layer(decoder_output[0])
predicted_token = logits.argmax(1)

In [ ]:
print(predicted_token)
print(tokenizer.convert_ids_to_tokens(predicted_token))

tensor([133])
['The']


Getting beam scores for each generated token:

In [ ]:
import torch
import torch.nn as nn

#summary_ids = model.generate(inputs["input_ids"], num_beams=6, min_length=0, max_length=40)
for i in range(1):
    out = model.generate(inputs["input_ids"], min_length=0, max_length=40, num_beams=2, num_return_sequences=1, output_scores=True, return_dict_in_generate=True, output_hidden_states=True)
    outscores = torch.cat([out['scores'][i].unsqueeze(1) for i in range(len(out['scores']))], dim=1)
    print("outscores:", outscores)
    print("best beam is seq: ", out['beam_indices'][0][-1].item())
    num_beams, num_chars, char_dim = outscores.shape
    num_top = 5
    print('Beam indices')
    print(out['beam_indices'])
    for s in range(num_beams):
        for c in range(num_chars):
            probas = nn.functional.softmax(outscores[s,c,:], dim=-1)
            top_proba, top_idx = probas.topk(num_top, dim=-1)
            num_seqs = top_proba.shape[0]
            print("seq"+str(s)+"char"+str(c)+":","---".join([ str("{:0.1f}".format(p.item()*100))+" "+tokenizer.decode(top_idx[i]) for i,p in enumerate(top_proba)]))

    print("out",tokenizer.batch_decode(out['sequences'], skip_special_tokens=True))

outscores: tensor([[[-20.0130, -12.2692, -11.3902,  ..., -11.4181, -11.2926, -11.8808],
         [-17.7894, -14.1151, -10.4689,  ..., -15.0756, -14.5126, -14.9504],
         [-13.4767, -13.3466,  -9.3276,  ..., -13.6885, -13.7565, -13.5956],
         ...,
         [-15.1182, -12.9712,  -0.1237,  ..., -12.7374, -12.8215, -12.7531],
         [-25.3114, -13.1845,  -9.9702,  ..., -13.3340, -13.2977, -13.1668],
         [-20.3617, -13.2935,  -8.8127,  ..., -12.4126, -12.6354, -12.4597]],

        [[-20.0130, -12.2692, -11.3902,  ..., -11.4181, -11.2926, -11.8808],
         [-27.1198, -13.6085, -13.6146,  ..., -14.0192, -13.6254, -13.7197],
         [-21.7452, -15.2058, -12.5938,  ..., -14.9339, -15.3812, -15.7374],
         ...,
         [-25.1851, -13.2993, -11.6019,  ..., -13.1152, -13.3353, -13.1804],
         [-23.0841, -12.9680,  -9.3915,  ..., -13.0015, -13.1562, -12.8894],
         [-12.8962, -13.0373,  -0.1164,  ..., -12.8852, -13.0208, -12.9275]]])
best beam is seq:  -1
Beam indice

Format of generating summary with beam sores:

In [ ]:
summary_ids = model.generate(inputs["input_ids"],  num_beams=2, min_length=0, max_length=40)
l = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
s = ''.join(l)+"\n"
print(s)

The winners of Britain's Got Talent have been honoured at the Classical Brit Awards.



In [ ]:
summary_ids = model.generate(inputs["input_ids"] , num_beams=2, min_length=0, max_length=40, output_scores=True, return_dict_in_generate=True)
gen_sequences = summary_ids.sequences[:, inputs["input_ids"].shape[-1]:]

print(summary_ids.scores[0][1])
print(summary_ids.sequences)


tensor([-24.9079, -12.0405, -11.6512,  ..., -10.3614, -10.4520, -10.5488])
tensor([[    2,   133,  4251,     9,  1444,    18,  8432, 18436,    33,    57,
         16962,    23,     5, 39641,  9446,  4229,     4,     2]])


In [ ]:
myFile = open("scores.txt","a+")

In [ ]:
l = len(summary_ids.scores) #taking length of tensor for each beam scores
print(summary_ids['beam_indices'])
myFile.write(str(summary_ids['beam_indices'])+"\n")
for i in range(l):
  for j in range(2):
    x = summary_ids.scores[i][j] #[id position][first beam]
    #y = summary_ids.scores[i][1] #[id position][second beam]
    x = x.sort(dim = 0, descending=True)
    #y = y.sort(dim = 0, descending=True)
    a = x[1][0]
    #b = y[1][0]
    s = str(j)+'th beam token for '+str(i)+'th index: '+ tokenizer.decode(a)+ ", "+ str(x[0][0])
    myFile.write(s+"\n")
    #print(j,'th beam token: ', tokenizer.decode(a), ",", x[0][0]) #printing the top most token
    print(s)
    #print('second beam token: ', tokenizer.decode(b), ",", y[0][0])

tensor([[ 0,  0,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1]])
0th beam token for 0th index: The, tensor(-1.4873)
1th beam token for 0th index: The, tensor(-1.4873)
0th beam token for 1th index:  winners, tensor(-1.2228)
1th beam token for 1th index:  Men, tensor(-0.1805)
0th beam token for 2th index:  Al, tensor(-0.1053)
1th beam token for 2th index:  of, tensor(-0.1200)
0th beam token for 3th index: oud, tensor(-0.2199)
1th beam token for 3th index:  Britain, tensor(-0.8490)
0th beam token for 4th index:  have, tensor(-1.1510)
1th beam token for 4th index: 's, tensor(-0.1204)
0th beam token for 5th index:  Got, tensor(-0.0768)
1th beam token for 5th index:  won, tensor(-0.7537)
0th beam token for 6th index:  Talent, tensor(-0.0744)
1th beam token for 6th index:  the, tensor(-0.9841)
0th beam token for 7th index:  have, tensor(-0.5256)
1th beam token for 7th index:  album, tensor(-0.9411)
0th beam token for 8th index:  been, tensor(-0.8442)
1th beam token for 8th i

Plotting the scores for each beam

In [ ]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
x = summary_ids.scores[0][1]
x = x.sort(dim = 0, descending=True)
print(x[1][0])

tensor(133)


In [ ]:
import plotly.express as px

x = summary_ids.scores[0][1]
fig = px.line(x)
fig.show()